In [ ]:
## load train model

In [ ]:
## run sampling


In [ ]:
## gather simulation input, test following predict algorithms


In [2]:
## test Bsyesian optimization
from sklearn import metrics
from sklearn import datasets
import numpy as np
import matplotlib.pyplot as plt
import catboost as cb
from bayes_opt import BayesianOptimization


In [ ]:
class optimizationClass():
    # 初始化的时候把数据集传进来
    def __init__(self,df):
        self.df = df 
        
     # 定义一个评估指标的函数评估模型好坏   
    def reg_calculate(self,true, prediction): 
        mse = metrics.mean_squared_error(true, prediction)
        rmse = np.sqrt(mse)
        mae = metrics.mean_absolute_error(true, prediction)
        mape = np.mean(np.abs((true - prediction) / true)) * 100
        r2 = metrics.r2_score(true, prediction)
        rmsle = np.sqrt(metrics.mean_squared_log_error(true, prediction))
        # print("mse: {}, rmse: {}, mae: {}, mape: {}, r2: {}, rmsle: {}".format(mse, rmse, mae, mape, r2, rmsle))
        # return mse, rmse, mae, mape, r2, rmsle
        return {"mse": mse, "rmse": rmse, "mae": mae, "mape": mape, "r2": r2, "rmsle": rmsle}    
    
    # 优化函数 参数为你需要调参的 hyper Parameter 这里用catBoost作为样例
    def optimization_function(self,iterations,learning_rate,depth,l2_leaf_reg):
        parameterDict = {"iterations":int(iterations),"learning_rate": float(learning_rate), "depth": int(depth),
                         "l2_leaf_reg":float(l2_leaf_reg),
                         "task_type":"CPU","logging_level":"Silent"}
        CB_Regressor=cb.CatBoostRegressor(**parameterDict)
        CB_Regressor.fit(self.df["X_train"], self.df["Y_train"])
        Y_pre=CB_Regressor.predict(self.df["X_test"])
        resDict =self.reg_calculate(self.df["Y_test"],Y_pre)
        return resDict["r2"]
    
    # 定义一下模型
    def run(self, init_points=2,n_iter=3):
        cb_bo = BayesianOptimization(
        self.optimization_function,
        {'iterations': (200, 5000),
        'learning_rate': (1e-6, 1e-2),
        'depth': (2, 15),
        'l2_leaf_reg': (0, 5)}
        )
        cb_bo.maximize(
                        init_points=init_points,
                        n_iter=n_iter)
        print("Final result:", cb_bo.max)
#     


In [ ]:
Opt = optimizationClass(df = {"X_train":X_Train, "X_test":X_Test, "Y_train":Y_Train, "Y_test":Y_Test})
Opt.run()

In [ ]:
## grid search

In [12]:
# preprocess data
from pathlib import Path
import os
path = Path('/home/lizz_lab/cse30019698/project/colmena/multisite_/finetuning-surrogates/analysis/runs')
titles = ['sampling','simulation','training','inference']
def find_files_with_title(directory, title):
    result = {}
    base_level = str(directory).count(os.sep)
    for root, dirs, files in os.walk(directory):
        if root.count(os.sep) <= base_level+3:  # 只遍历三级子目录
            for file in files:
                if title in file:
                    relative_path = os.path.relpath(os.path.join(root, file), str(directory))
                    result[relative_path] = os.path.join(root, file)
    return result

def get_running_metrics():
    pass

simulation_data = find_files_with_title(path, 'simulation')
print(simulation_data)

{'20240124_014903/dft-md-24Jan23-174907-4d97c4/simulation-results.json': '/home/lizz_lab/cse30019698/project/colmena/multisite_/finetuning-surrogates/analysis/runs/20240124_014903/dft-md-24Jan23-174907-4d97c4/simulation-results.json', '20240124_001447/dft-md-24Jan23-161451-5b14aa/simulation-results.json': '/home/lizz_lab/cse30019698/project/colmena/multisite_/finetuning-surrogates/analysis/runs/20240124_001447/dft-md-24Jan23-161451-5b14aa/simulation-results.json', '20240124_174540/dft-md-24Jan24-094544-8a6f62/simulation-results.json': '/home/lizz_lab/cse30019698/project/colmena/multisite_/finetuning-surrogates/analysis/runs/20240124_174540/dft-md-24Jan24-094544-8a6f62/simulation-results.json', '20240220_172152/dft-md-24Feb20-092155-f38847/simulation-results.json': '/home/lizz_lab/cse30019698/project/colmena/multisite_/finetuning-surrogates/analysis/runs/20240220_172152/dft-md-24Feb20-092155-f38847/simulation-results.json', '20240221_151441/dft-md-24Feb21-071447-e22b54/simulation-result

In [ ]:
## XGBoost
## test random forest
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# 读取数据集
data = pd.read_csv('task_runtimes.csv')

# 提取特征和目标变量
X = data.drop('runtime', axis=1)  # 特征
y = data['runtime']  # 目标变量

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 创建随机森林回归模型
rf = RandomForestRegressor(n_estimators=100, random_state=42)

# 训练模型
rf.fit(X_train, y_train)

# 在测试集上进行预测
y_pred = rf.predict(X_test)

# 评估模型性能
mse = mean_squared_error(y_test, y_pred)
print(f"均方误差（MSE）: {mse}")